In [ ]:
# Ch09-3 Geometric Operations

In [ ]:
# Retrieve data
from Bio import PDB 
repository = PDB.PDBList() 
parser = PDB.PDBParser() 
repository.retrieve_pdb_file('1TUP', pdir='.', file_format='pdb') 
p53_1tup = parser.get_structure('P 53', 'pdb1tup.ent') 

In [ ]:
# Recall residue types
my_residues = set() 
for residue in p53_1tup.get_residues(): 
    my_residues.add(residue.id[0]) 
print(my_residues) 

In [ ]:
# Compute masses for chains, zincs, and waters

import numpy as np

# Function to compute mass
def get_mass(atoms, accept_fun=lambda x: True):
    return sum([atom.mass for atom in atoms if accept_fun(atom)])

# Extract chain names
chain_names = [chain.id for chain in p53_1tup.get_chains()]

# Initialize NumPy array for masses
my_mass = np.ndarray((len(chain_names), 3))

# Iterate over chains to compute mass
for i, chain in enumerate(p53_1tup.get_chains()):
    my_mass[i, 0] = get_mass(chain.get_atoms())

print("Mass array:", my_mass)


In [ ]:
# Compute geometric center and center of mass
def get_center(atoms, 
    weight_fun=lambda atom: 1 if atom.parent.id[0] != 'W' else 0): 
    xsum = ysum = zsum = 0.0 
    acum = 0.0 
    for atom in atoms: 
        x, y, z = atom.coord 
        weight = weight_fun(atom) 
        acum += weight 
        xsum += weight * x 
        ysum += weight * y 
        zsum += weight * z 
    return xsum / acum, ysum / acum, zsum / acum 
print(get_center(p53_1tup.get_atoms())) 
print(get_center(p53_1tup.get_atoms(), 
    weight_fun=lambda atom: atom.mass if atom.parent.id[0] != 'W' else 0)) 

In [ ]:
# Compute center of mass and geometric center of each chain
import pandas as pd
my_center = np.ndarray((len(chain_names), 6)) 
for i, chain in enumerate(p53_1tup.get_chains()): 
    x, y, z = get_center(chain.get_atoms()) 
    my_center[i, 0] = x 
    my_center[i, 1] = y 
    my_center[i, 2] = z 
    x, y, z = get_center(chain.get_atoms(), 
        weight_fun=lambda atom: atom.mass if atom.parent.id[0] != 'W' else 0) 
    my_center[i, 3] = x 
    my_center[i, 4] = y 
    my_center[i, 5] = z 
weights = pd.DataFrame(my_center, index=chain_names, 
    columns=['X', 'Y', 'Z', 'X (Mass)', 'Y (Mass)', 'Z (Mass)']) 
print(weights) 

In [ ]:
## End of Notebook ##